In [277]:
from estimator.estimator import *
from utils import *

In [278]:
# LWE
n1 = 512
q1 = 2048

lwe1_key_type = "Binary"
lwe1_noise_stddev = 0.8293
lwe1_key_distribution = ND.SparseBinary(n1 // 2, n1)

# lwe1_key_type = "Ternary"
# lwe1_noise_stddev = 0.63
# lwe1_key_distribution = ND.SparseTernary(n1 // 4, n1 // 4, n1)

In [279]:
check_security_with_data(
    n1, q1, lwe1_key_type, lwe1_key_distribution, lwe1_noise_stddev
)

shape: (1, 7)
┌───────────┬─────────┬──────────────┬───────────────┬──────────────┬───────────────┬──────────────┐
│ dimension ┆ modulus ┆ secret_distr ┆ secret_stddev ┆ noise_stddev ┆ security_bits ┆ datetime     │
│ ---       ┆ ---     ┆ ---          ┆ ---           ┆ ---          ┆ ---           ┆ ---          │
│ u64       ┆ u64     ┆ str          ┆ f64           ┆ f64          ┆ f64           ┆ datetime[μs] │
╞═══════════╪═════════╪══════════════╪═══════════════╪══════════════╪═══════════════╪══════════════╡
│ 512       ┆ 2048    ┆ Binary       ┆ 0.5           ┆ 0.8293       ┆ 128.024617    ┆ 2025-02-28   │
│           ┆         ┆              ┆               ┆              ┆               ┆ 14:36:04.107 │
│           ┆         ┆              ┆               ┆              ┆               ┆ 401          │
└───────────┴─────────┴──────────────┴───────────────┴──────────────┴───────────────┴──────────────┘


In [280]:
# BSK 1
N1 = 1024
Q1 = Q27_10

# bsk1_key_type = "Binary"
# bsk1_stddev = 4.49
# bsk1_key_distribution = ND.SparseBinary(N1 // 2, N1)

bsk1_key_type = "Ternary"
bsk1_stddev = 3.1859
bsk1_key_distribution = ND.SparseTernary(N1 // 4, N1 // 4, N1)

In [281]:
check_security_with_data(N1, Q1, bsk1_key_type, bsk1_key_distribution, bsk1_stddev)

shape: (1, 7)
┌───────────┬───────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┐
│ dimension ┆ modulus   ┆ secret_distr ┆ secret_stdde ┆ noise_stddev ┆ security_bit ┆ datetime     │
│ ---       ┆ ---       ┆ ---          ┆ v            ┆ ---          ┆ s            ┆ ---          │
│ u64       ┆ u64       ┆ str          ┆ ---          ┆ f64          ┆ ---          ┆ datetime[μs] │
│           ┆           ┆              ┆ f64          ┆              ┆ f64          ┆              │
╞═══════════╪═══════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╡
│ 1024      ┆ 134215681 ┆ Ternary      ┆ 0.707107     ┆ 3.1859       ┆ 128.029066   ┆ 2025-02-28   │
│           ┆           ┆              ┆              ┆              ┆              ┆ 14:37:12.181 │
│           ┆           ┆              ┆              ┆              ┆              ┆ 723          │
└───────────┴───────────┴──────────────┴──────────────┴──────────────┴───────

In [282]:
n2 = 670

ksk_type = "Binary"
ksk_noise_stddev = 2.0329 * (2**10)
ksk_distribution = ND.SparseBinary(n2 // 2, n2)

# ksk_type = "Ternary"
# ksk_noise_stddev = 3.2
# ksk_distribution = ND.SparseTernary(n2 // 4, n2 // 4, n2)

In [283]:
check_security_with_data(n2, Q1, ksk_type, ksk_distribution, ksk_noise_stddev)

shape: (1, 7)
┌───────────┬───────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┐
│ dimension ┆ modulus   ┆ secret_distr ┆ secret_stdde ┆ noise_stddev ┆ security_bit ┆ datetime     │
│ ---       ┆ ---       ┆ ---          ┆ v            ┆ ---          ┆ s            ┆ ---          │
│ u64       ┆ u64       ┆ str          ┆ ---          ┆ f64          ┆ ---          ┆ datetime[μs] │
│           ┆           ┆              ┆ f64          ┆              ┆ f64          ┆              │
╞═══════════╪═══════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╡
│ 670       ┆ 134215681 ┆ Binary       ┆ 0.5          ┆ 2081.6896    ┆ 128.021958   ┆ 2025-02-28   │
│           ┆           ┆              ┆              ┆              ┆              ┆ 14:38:34.927 │
│           ┆           ┆              ┆              ┆              ┆              ┆ 018          │
└───────────┴───────────┴──────────────┴──────────────┴──────────────┴───────

In [284]:
N2 = 2048
Q2 = Q50_11

# bsk2_key_type = "Binary"
# bsk2_nosie_stddev = 4.63
# bsk2_key_distribution = ND.SparseBinary(N2 // 2, N2)

bsk2_key_type = "Ternary"
bsk2_nosie_stddev = 0.3908
bsk2_key_distribution = ND.SparseTernary(N2 // 4, N2 // 4, N2)

In [285]:
check_security_with_data(
    N2, Q2, bsk2_key_type, bsk2_key_distribution, bsk2_nosie_stddev
)

shape: (1, 7)
┌───────────┬──────────────┬──────────────┬──────────────┬─────────────┬─────────────┬─────────────┐
│ dimension ┆ modulus      ┆ secret_distr ┆ secret_stdde ┆ noise_stdde ┆ security_bi ┆ datetime    │
│ ---       ┆ ---          ┆ ---          ┆ v            ┆ v           ┆ ts          ┆ ---         │
│ u64       ┆ u64          ┆ str          ┆ ---          ┆ ---         ┆ ---         ┆ datetime[μs │
│           ┆              ┆              ┆ f64          ┆ f64         ┆ f64         ┆ ]           │
╞═══════════╪══════════════╪══════════════╪══════════════╪═════════════╪═════════════╪═════════════╡
│ 2048      ┆ 112589990682 ┆ Ternary      ┆ 0.707107     ┆ 0.3908      ┆ 128.000375  ┆ 2025-02-28  │
│           ┆ 6241         ┆              ┆              ┆             ┆             ┆ 14:41:14.67 │
│           ┆              ┆              ┆              ┆             ┆             ┆ 7156        │
└───────────┴──────────────┴──────────────┴──────────────┴─────────────┴─────